In [ ]:
import pandas as pd
from Q2Funcs import find_uniques

df_movies = pd.read_csv('data/movie.csv')
df_movies.head()

# Ongewenste kolommen verwijderen
df_movies.drop(["movie_imdb_link", "aspect_ratio"], axis=1, inplace=True)

#Onduidelijke kolomnamen aanpassen
df_movies.rename(columns={'color': 'Colour',
                          'director_name': 'Director',
                          'num_critic_for_reviews': 'Number of critics',
                          'duration': 'Duration',
                          'director_facebook_likes': 'Director FB likes',
                          'actor_3_facebook_likes': 'Actor 3 FB likes',
                          'actor_2_name': 'Actor 2 name',
                          'actor_1_facebook_likes': 'Actor 1 FB likes',
                          'gross': 'Gross',
                          'genres': 'Genres',
                          'actor_1_name': 'Actor 1 name',
                          'movie_title': 'Movie title',
                          'num_voted_users': 'Number of voted users',
                          'cast_total_facebook_likes': 'Total Cast FB likes',
                          'actor_3_name': 'Actor 3 name',
                          'facenumber_in_poster': 'Number of faces on poster',
                          'plot_keywords': 'Plot Keywords',
                          'num_user_for_reviews': 'Number of user reviews',
                          'language': 'Language',
                          'country': 'Country',
                          'content_rating': 'Age rating',
                          'budget': 'Budget',
                          'title_year': 'Release year',
                          'actor_2_facebook_likes': 'Actor 2 FB likes',
                          'imdb_score': 'IMDB Score',
                          'movie_facebook_likes': 'Movie FB likes'}, inplace=True)

# Volgorde kolommen aanpassen
df_movies = df_movies[['Movie title',
                       'Release year',
                       'Director',
                       'Director FB likes',
                       'Gross',
                       'Budget',
                       'Duration',
                       'Language',
                       'Country',
                       'Colour',
                       'Genres',
                       'IMDB Score',
                       'Number of voted users',
                       'Number of critics',
                       'Number of user reviews',
                       'Age rating',
                       'Total Cast FB likes',
                       'Movie FB likes',
                       'Actor 1 name',
                       'Actor 2 name',
                       'Actor 3 name',
                       'Actor 1 FB likes',
                       'Actor 2 FB likes',
                       'Actor 3 FB likes',
                       'Plot Keywords',
                       'Number of faces on poster',
                       ]]

# Datatypen aanpassen
# 1. Floats omzetten naar integers
#  De dataset bevat geen kolommen die dienen te worden bewaard als float, behalve `IMDB Score`
df_movies_IMDB_Score = df_movies["IMDB Score"]  # Tijdelijke kopie van de kolom `IMDB Score`
df_movies = df_movies.drop('IMDB Score', axis=1).fillna(0).astype(int, errors='ignore') # Waarden omzetten naar integers
df_movies.insert(11, "IMDB Score", df_movies_IMDB_Score)  # `IMDB Score` weer toevoegen aan originele DataFrame
del df_movies_IMDB_Score

# 2. De kolom `Release year` omzettten van integers naar het datetime-datatype
df_movies["Release year"] = pd.to_datetime(df_movies["Release year"], format='%Y', errors='coerce')



## 3. Data Cleaning


In [ ]:
# NaN-types verwijderen
df_movies.dropna(inplace=True)

# Dubbele titels verwijderen
df_movies.sort_values("Release year", inplace=True)  # Sorteren op uitgavejaar
df_movies.drop_duplicates(subset="Movie title", keep="last", inplace=True)  # Alleen meest recente versie blijft bewaard

# Negatieve waardes verwijderen
num = df_movies._get_numeric_data()
num[num < 0] = 0

# Z-toets
Hieronder wordt de volgende Z-toets uitgevoerd:
> Een filmcriticus stelt dat de score van engelstalige films lager is dan gemiddeld.
Onderzoek met de dataset of deze filmcriticus gelijk heeft. Neem een steekproef van 100 engelstalige films en beschouw de hele dataset als populatie. Neem als betrouwbaarheid 90%. Gebruik van de dataset alleen de filmgegevens waarbij zowel de taal `Language` als de score `IMDB Score` bekend zijn.

Het gaat hierbij om linkszijdig testen.

H<sub>0</sub> = Engelstalige films scoren gemiddeld niet lager dan gemiddeld.  
H<sub>1</sub> = Engelstalige films scoren gemiddeld lager dan gemiddeld.

$\alpha = 0.10$  


$Z_{\alpha} = 1.64$ 


In [ ]:
import scipy.stats as st
z_alpha = 1.64
z_alpha

Steekproef nemen:

In [ ]:
df_movies.head()
df_ztest = df_movies.loc[df_movies["Language"] == "English", ["IMDB Score"]].sample(100)
df_ztest.head()

Gemiddelde en standaardverdeling van steekproef en gemiddelde van populatie berekenen:

In [ ]:
x_bar = df_ztest["IMDB Score"].mean()
s_eng = df_ztest["IMDB Score"].std()

mu = df_movies["IMDB Score"].mean()

print(f"Sample:\n \tMean: {x_bar}\n\tStandard Deviation: {s_eng}")
print(f"Population:\n\tMean: {mu}")

Standaard fout berekenen:

In [ ]:
SE = s_eng / (len(df_ztest)**0.5)
print(f"Standard Error: {SE}")

Z score berekenen:

In [ ]:
z = (x_bar - mu) / SE
print(f"Z: {z}")

Hypothese verwerpen of accepteren:

In [ ]:
if z > z_alpha:
    print("Verwerp H_0, accepteer H_1")
else:
    print("Geen duidelijke conclusie; blijf skeptisch!")